# Duplicate User Data for Preference Demo

The purpose of this notebook is to create user data for Ryan Holloway
that duplicates the data attributed to Jane. This duplicated data will
be used to demonstrate how user preferences may change the summary output.

In [1]:
from google.cloud import secretmanager
import pandas as pd
import numpy as np
from currensee.utils.db_utils import create_pg_engine
from sqlalchemy import text

import pandas as pd
import random
import os
import re
from faker import Faker


2025-07-08 13:26:33,493 - currensee.core.secrets - INFO - PROJECT_ID not found, attempting to load from .env file
2025-07-08 13:26:33,498 - currensee.core.secrets - INFO - Loaded environment from ./.env
2025-07-08 13:26:33,499 - currensee.core.secrets - INFO - SecretManager initialized with project_id: adsp-34002-on02-sopho-scribe


## Define the Connection & Load Data

In [2]:
# Define DB_NAME
DB_NAME = "crm_outlook"

In [3]:
engine = create_pg_engine(db_name=DB_NAME)

In [6]:
# meetings between jane and her clients 
# have times be after 2023
meeting_df = pd.read_sql("SELECT * FROM meeting_data", con=engine)
meeting_df.head()

,meeting_timestamp,host,host_email,invitees,invitee_emails,meeting_subject
0,2018-07-17 15:00:00,Jane Moneypenny,jane.moneypenny1@outlook.com,Cynthia Hobbs,cynthia.hobbs@abbvie.com,AbbVie - Introductory Call & Bankwell Services...
1,2018-07-24 14:00:00,Jane Moneypenny,jane.moneypenny1@outlook.com,Jennifer Phelps,jennifer.phelps@aerovironment.com,AeroVironment - Initial Discussion on Financia...
2,2018-08-07 10:00:00,Jane Moneypenny,jane.moneypenny1@outlook.com,Kyle Waters,kyle.waters@amedisys.com,Amedisys - Discussing Cash Management Needs
3,2018-08-15 11:00:00,Jane Moneypenny,jane.moneypenny1@outlook.com,Kyle Waters,kyle.waters@amedisys.com,Amedisys - Cash Concentration Services Demo
4,2018-08-29 14:00:00,Jane Moneypenny,jane.moneypenny1@outlook.com,Denise Moore,denise.moore@celestica.com,Celestica - Initial Discussion on Liquidity Ma...


In [7]:
# emails between jane and her clients
email_df = pd.read_sql("SELECT * FROM email_data", con=engine)
email_df.head()

,email_timestamp,to_names,to_emails,from_name,from_email,email_subject,email_body
0,2018-07-15 09:30:00,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Jane Moneypenny,jane.moneypenny1@outlook.com,Introduction - Bankwell Financial Services for...,"Hi Cynthia,\n\nMy name is Jane Moneypenny, and..."
1,2018-07-16 11:05:00,Jane Moneypenny,jane.moneypenny1@outlook.com,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Re: Introduction - Bankwell Financial Services...,"Hi Jane,\n\nThank you for reaching out. AbbVie..."
2,2018-07-16 11:15:00,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Jane Moneypenny,jane.moneypenny1@outlook.com,Re: Introduction - Bankwell Financial Services...,"Hi Cynthia,\n\nTuesday at 2 PM CT works perfec..."
3,2018-08-01 14:00:00,Jennifer Phelps,jennifer.phelps@aerovironment.com,Jane Moneypenny,jane.moneypenny1@outlook.com,Follow Up: AeroVironment Financial Needs,"Hi Jennifer,\n\nIt was a pleasure speaking wit..."
4,2018-08-02 10:15:00,Jane Moneypenny,jane.moneypenny1@outlook.com,Jennifer Phelps,jennifer.phelps@aerovironment.com,Re: Follow Up: AeroVironment Financial Needs,"Hi Jane,\n\nThanks for sending this over. The ..."


## Duplicate the Data for Ryan

In [16]:
# For the employee Ryan Holloway, duplicate all messages between Jane and the client 
# as if Jane forwarded the messages to him when he was hired
ryan_meeting_df = meeting_df.copy()
ryan_meeting_df.loc[ryan_meeting_df['host'] == 'Jane Moneypenny', 'host'] = 'Ryan Holloway'
ryan_meeting_df.loc[ryan_meeting_df['host'] == 'Ryan Holloway', 'host_email'] = 'ryan.holloway@bankwell.com'

In [17]:
ryan_meeting_df.head()

,meeting_timestamp,host,host_email,invitees,invitee_emails,meeting_subject
0,2018-07-17 15:00:00,Ryan Holloway,ryan.holloway@bankwell.com,Cynthia Hobbs,cynthia.hobbs@abbvie.com,AbbVie - Introductory Call & Bankwell Services...
1,2018-07-24 14:00:00,Ryan Holloway,ryan.holloway@bankwell.com,Jennifer Phelps,jennifer.phelps@aerovironment.com,AeroVironment - Initial Discussion on Financia...
2,2018-08-07 10:00:00,Ryan Holloway,ryan.holloway@bankwell.com,Kyle Waters,kyle.waters@amedisys.com,Amedisys - Discussing Cash Management Needs
3,2018-08-15 11:00:00,Ryan Holloway,ryan.holloway@bankwell.com,Kyle Waters,kyle.waters@amedisys.com,Amedisys - Cash Concentration Services Demo
4,2018-08-29 14:00:00,Ryan Holloway,ryan.holloway@bankwell.com,Denise Moore,denise.moore@celestica.com,Celestica - Initial Discussion on Liquidity Ma...


In [21]:
ryan_email_df = email_df.copy()
ryan_email_df.loc[ryan_email_df['from_name'] == 'Jane Moneypenny', 'from_name'] = 'Ryan Holloway'
ryan_email_df.loc[ryan_email_df['to_names'] == 'Jane Moneypenny', 'to_names'] = 'Ryan Holloway'
ryan_email_df.loc[ryan_email_df['from_name'] == 'Ryan Holloway', 'from_email'] = 'ryan.holloway@bankwell.com'
ryan_email_df.loc[ryan_email_df['to_names'] == 'Ryan Holloway', 'to_emails'] = 'ryan.holloway@bankwell.com'

In [22]:
ryan_email_df.head()

,email_timestamp,to_names,to_emails,from_name,from_email,email_subject,email_body
0,2018-07-15 09:30:00,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Ryan Holloway,ryan.holloway@bankwell.com,Introduction - Bankwell Financial Services for...,"Hi Cynthia,\n\nMy name is Jane Moneypenny, and..."
1,2018-07-16 11:05:00,Ryan Holloway,ryan.holloway@bankwell.com,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Re: Introduction - Bankwell Financial Services...,"Hi Jane,\n\nThank you for reaching out. AbbVie..."
2,2018-07-16 11:15:00,Cynthia Hobbs,cynthia.hobbs@abbvie.com,Ryan Holloway,ryan.holloway@bankwell.com,Re: Introduction - Bankwell Financial Services...,"Hi Cynthia,\n\nTuesday at 2 PM CT works perfec..."
3,2018-08-01 14:00:00,Jennifer Phelps,jennifer.phelps@aerovironment.com,Ryan Holloway,ryan.holloway@bankwell.com,Follow Up: AeroVironment Financial Needs,"Hi Jennifer,\n\nIt was a pleasure speaking wit..."
4,2018-08-02 10:15:00,Ryan Holloway,ryan.holloway@bankwell.com,Jennifer Phelps,jennifer.phelps@aerovironment.com,Re: Follow Up: AeroVironment Financial Needs,"Hi Jane,\n\nThanks for sending this over. The ..."


## Combine the Data and Store in DB

In [23]:
combined_meeting_df = pd.concat([meeting_df,ryan_meeting_df], ignore_index=True)
combined_email_df = pd.concat([email_df,ryan_email_df], ignore_index=True)